In [1]:
import tensorflow as tf
import keras as kr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from functools import reduce

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, Input

%matplotlib inline  

Using TensorFlow backend.


In [11]:
NUMBER_SIZE = 15
THRESHOLD_COUNT = 5

BATCH_NUMBER_COUNT_50 = 50
SINGLE_SAMPLE_SIZE_50 = BATCH_NUMBER_COUNT_50 * NUMBER_SIZE
INPUT_DIM = SINGLE_SAMPLE_SIZE_50

CLASSES = { "none": 0, "attack": 1 }

In [12]:
def create_single_sample_from_array(numbers):
  # Treat the list of phone numbers as a single huge list digits
  # pretty much like greyscale images, etc.
  digit_str = ''.join([str(i) for i in numbers])
  # Now create an image-like array
  return list(digit_str)

def create_single_sample_from_dataframe(dataframe_row):
  return create_single_sample_from_array(np.array(dataframe_row))

def combine_dfs(dfs, outcome_values=None, shuffle=True):
  # Combine data with sequence with outcome
  df_list = []
  for idx, df in enumerate(dfs):
    data_arr = []
    for index, series in df.iterrows():
      series_value = [v for _, v in series.iteritems()]
      combined_series = create_single_sample_from_array(series_value)
      data_arr.append(combined_series)
    
    df_arr_row_count = len(data_arr)
    
    outcome_value = outcome_values[idx] if outcome_values else idx
    data_outcome_tuples = zip(data_arr, [outcome_value] * df_arr_row_count)

    # DEBUG
    print("outcome_value:%s of all possible outcome_values:%s" % (outcome_value, outcome_values))

    df_list += data_outcome_tuples   
    
  # Merge data with sequence and no sequence
  dataframe = pd.DataFrame(df_list, columns=["sample","outcome"])
  # Shuffle (or rather randomly select samples) but 1.0 means all
  df_random = dataframe.sample(frac=1) if shuffle else dataframe
  return df_random

def extract_sample_and_outcome(df, sample_col_name='sample', outcome_col_name='outcome'):
  X = [i for i in df[sample_col_name]]
  Y = [i for i in df[outcome_col_name]]
  return (X, Y)

def load_data(filename, expected_shape_tuple):
  data = pd.read_csv(filename, header=None)
  print("data file:%s, data.shape (should be %s): %s" % (filename, expected_shape_tuple, data.shape))
  if data.shape != expected_shape_tuple:
    raise ValueError("data.shape:%s does not match excpected shape:%s" % (data.shape, expected_shape_tuple))
  return data
    
def prepare_data(filenames, expected_shape_tuples, outcome_values=None, shuffle=True):
  dfs = []
  for idx, filename in enumerate(filenames):
    dfs.append(load_data(filename, expected_shape_tuples[idx]))

  df_all = combine_dfs(dfs, outcome_values, shuffle)
  if (df_all.shape[0] != reduce((lambda m, i: m + i.shape[0]), dfs,0)):
    raise ValueError("There is a problem with combine_dfs. df_all.shape:%s does not match the sum of df_seq:%s and df_no_seq:%s" % (df_all.shape, df_seq.shape, df_no_seq.shape))
  return df_all

def prepare_train_data(dataframe, single_sample_size):
  df_row_count = dataframe.shape[0]
  (X, Y) = extract_sample_and_outcome(dataframe)
  X_train = np.array(X)
  Y_train = np.array(Y)

  print("type(X):%s, len(X):%d, len(X[0]):%d" % (type(X), len(X), len(X[0])))
  print("df_row_count: %d" % df_row_count)
  print("single_sample_size_bit:%d" % single_sample_size)
  print("X_train.shape:%s, Y_train.shape:%s" % (X_train.shape, Y_train.shape))
  print("\n")

  print("type(Y[0]):%s, Y[0]:%s" % (type(Y[0]), Y[0]))

  return (X_train, Y_train)

In [13]:
# Load data from files into dataframe

df_random_2040_2040_combo_sample_number_50 = prepare_data(['data_no_sequence_2040_sample_number_50.csv','data_sequence_sparse_head_heavy_tail_heavy_ooo_mid_combo_2040_sample_number_50.csv'], [(2040,50), (2040,50)], [CLASSES["none"],CLASSES["attack"]]) 

df_random_10200_10200_combo_sample_number_50 = prepare_data(['data_no_sequence_10200_sample_number_50.csv','data_sequence_sparse_head_heavy_tail_heavy_ooo_mid_combo_10200_sample_number_50.csv'], [(10200,50), (10200,50)], [CLASSES["none"],CLASSES["attack"]]) 

data file:data_no_sequence_2040_sample_number_50.csv, data.shape (should be (2040, 50)): (2040, 50)
data file:data_sequence_sparse_head_heavy_tail_heavy_ooo_mid_combo_2040_sample_number_50.csv, data.shape (should be (2040, 50)): (2040, 50)
outcome_value:0 of all possible outcome_values:[0, 1]
outcome_value:1 of all possible outcome_values:[0, 1]
data file:data_no_sequence_10200_sample_number_50.csv, data.shape (should be (10200, 50)): (10200, 50)
data file:data_sequence_sparse_head_heavy_tail_heavy_ooo_mid_combo_10200_sample_number_50.csv, data.shape (should be (10200, 50)): (10200, 50)
outcome_value:0 of all possible outcome_values:[0, 1]
outcome_value:1 of all possible outcome_values:[0, 1]


In [14]:
# SANITY CHECK #1

# There should be 'sample' and 'outcome' columns
# 'sample' contains all phone numbers in period broken down into individual digits
# and combined
# 'outcome' is 0 or 1 for binary
# Also there should be mixed 0 and 1 since the samples have been randomize
df_random_2040_2040_combo_sample_number_50.tail()

,sample,outcome
978,"[2, 9, 3, 2, 8, 4, 3, 2, 7, 5, 2, 5, 0, 7, 2, ...",0
2941,"[8, 8, 7, 4, 1, 1, 9, 4, 1, 3, 4, 7, 6, 8, 5, ...",1
2439,"[8, 0, 9, 6, 5, 3, 3, 2, 0, 6, 4, 7, 2, 2, 5, ...",1
2983,"[6, 7, 9, 2, 4, 4, 0, 2, 1, 3, 7, 7, 5, 6, 1, ...",1
1517,"[3, 8, 1, 2, 7, 4, 6, 1, 8, 8, 3, 3, 4, 7, 5, ...",0


In [15]:
# SANITY CHECK #2

# 50% or 2040 samples should be 1
np.count_nonzero(df_random_2040_2040_combo_sample_number_50['outcome'])

2040

In [16]:
# SANITY CHECK #3

# The 'sample' length should be same as SINGLE_SAMPLE_SIZE_50 = BATCH_NUMBER_COUNT_50 * NUMBER_SIZE
len(df_random_2040_2040_combo_sample_number_50['sample'][1])

750

In [17]:
# Split out dataframe containing both samples and outcome

(X_2040_2040_combo_sample_number_50_train, Y_2040_2040_combo_sample_number_50_train) = prepare_train_data(df_random_2040_2040_combo_sample_number_50, SINGLE_SAMPLE_SIZE_50)

(X_10200_10200_combo_sample_number_50_train, Y_10200_10200_combo_sample_number_50_train) = prepare_train_data(df_random_10200_10200_combo_sample_number_50, SINGLE_SAMPLE_SIZE_50)

type(X):<class 'list'>, len(X):4080, len(X[0]):750
df_row_count: 4080
single_sample_size_bit:750
X_train.shape:(4080, 750), Y_train.shape:(4080,)


type(Y[0]):<class 'numpy.int64'>, Y[0]:0
type(X):<class 'list'>, len(X):20400, len(X[0]):750
df_row_count: 20400
single_sample_size_bit:750
X_train.shape:(20400, 750), Y_train.shape:(20400,)


type(Y[0]):<class 'numpy.int64'>, Y[0]:0


In [18]:
# SANITY CHECK #4

# The total rows should be attack and non-attack rows
# The wide should be SINGLE_SAMPLE_SIZE_50
X_2040_2040_combo_sample_number_50_train.shape

(4080, 750)

In [19]:
# SANITY CHECK #5

# The total rows should be attack and non-attack rows
# The wide should be single digit, i.e., 0 or 1
Y_2040_2040_combo_sample_number_50_train.shape

(4080,)

In [20]:
basic_model = Sequential()
basic_model.add(Dense(100, input_dim=INPUT_DIM, activation="relu"))
basic_model.add(Dense(64, activation="relu"))
basic_model.add(Dense(1, activation="sigmoid"))

basic_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
basic_model.fit(X_10200_10200_combo_sample_number_50_train, Y_10200_10200_combo_sample_number_50_train, validation_split=0.2, epochs=50)

Train on 16320 samples, validate on 4080 samples
Epoch 1/50
16320/16320 [==============================] - 5s - loss: 0.7315 - acc: 0.4988 - val_loss: 0.6953 - val_acc: 0.5047
Epoch 2/50
16320/16320 [==============================] - 5s - loss: 0.7014 - acc: 0.4997 - val_loss: 0.6941 - val_acc: 0.5034
Epoch 3/50
16320/16320 [==============================] - 5s - loss: 0.6957 - acc: 0.4998 - val_loss: 0.6931 - val_acc: 0.5049
Epoch 4/50
16320/16320 [==============================] - 5s - loss: 0.6932 - acc: 0.5001 - val_loss: 0.6932 - val_acc: 0.4958
Epoch 5/50
16320/16320 [==============================] - 5s - loss: 0.6932 - acc: 0.4963 - val_loss: 0.6932 - val_acc: 0.4958
Epoch 6/50
16320/16320 [==============================] - 5s - loss: 0.6931 - acc: 0.5022 - val_loss: 0.6931 - val_acc: 0.5051
Epoch 7/50
16320/16320 [==============================] - 5s - loss: 0.6932 - acc: 0.4947 - val_loss: 0.6931 - val_acc: 0.5051
Epoch 8/50
16320/16320 [==============================] - 5s -

In [21]:
predict_arr = basic_model.predict_classes(X_2040_2040_combo_sample_number_50_train)
# Each element in 'a' will contain 1/True if prediction matches expected outcome
a = (predict_arr == np.array([[r] for r in Y_2040_2040_combo_sample_number_50_train]))
# % of correct predictions
np.count_nonzero(a)/np.size(a)

4064/4080 [============================>.] - ETA: 0s

0.5

In [22]:
# Other models that we tested but did not have improved results

def create_baseline():
  # create model
  model = Sequential()
  model.add(Dense(100, input_dim=INPUT_DIM, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_model_A():
  # create model
  model = Sequential()
  model.add(Dense(100, input_dim=INPUT_DIM, kernel_initializer='normal', activation='relu'))
  model.add(Dense(64, activation="relu"))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_model_B():
  # create model
  model = Sequential()
  model.add(Dense(50, input_dim=INPUT_DIM, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_model_C():
  # create model
  model = Sequential()
  model.add(Dense(50, input_dim=INPUT_DIM, kernel_initializer='normal', activation='relu'))
  model.add(Dense(32, activation="relu"))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_model_D():
  # create model
  model = Sequential()
  model.add(Dense(5*NUMBER_SIZE_BIT, input_dim=INPUT_DIM, kernel_initializer='normal', activation='relu'))
  model.add(Dense(NUMBER_SIZE_BIT, activation="relu"))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_model_E():
  # create model
  model = Sequential()
  model.add(Dense(THRESHOLD_COUNT*NUMBER_SIZE_BIT, input_dim=INPUT_DIM, kernel_initializer='normal', activation='relu'))
  model.add(Dense(THRESHOLD_COUNT, activation="relu"))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_model_F():
  # create model
  model = Sequential()
  model.add(Dense(INPUT_DIM, input_dim=INPUT_DIM, kernel_initializer='normal', activation='relu'))
  model.add(Dense(BATCH_NUMBER_COUNT, activation="relu"))
  model.add(Dense(THRESHOLD_COUNT, activation="relu"))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_model_G():
  # create model
  model = Sequential()
  model.add(Dense(INPUT_DIM, input_dim=INPUT_DIM, kernel_initializer='normal', activation='relu'))
  model.add(Dense(THRESHOLD_COUNT*NUMBER_SIZE, activation="relu"))
  model.add(Dense(THRESHOLD_COUNT, activation="relu"))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_model_F_1():
  # create model
  model = Sequential()
  model.add(Dense(INPUT_DIM, input_dim=INPUT_DIM, kernel_initializer='normal', activation='relu'))
  model.add(Dense(BATCH_NUMBER_COUNT, activation="relu"))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_model_H():
  # create model
  model = Sequential()
  model.add(Dense(BATCH_NUMBER_COUNT, input_dim=INPUT_DIM, kernel_initializer='normal', activation='relu'))
  model.add(Dense(THRESHOLD_COUNT, activation="relu"))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_model_I():
  # create model
  model = Sequential()
  model.add(Dense(BATCH_NUMBER_COUNT, input_dim=INPUT_DIM, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [15]:
# You can evaluate other models that we created but the results do NOT improve

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

# Evaluate baseline model
seed = 7
estimator = KerasClassifier(build_fn=create_baseline, epochs=10, batch_size=5, verbose=1)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X_10200_10200_combo_sample_number_50_train, Y_10200_10200_combo_sample_number_50_train, cv=kfold)

print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/10
16320/16320 [==============================] - 14s - loss: 0.6997 - acc: 0.5002    
Epoch 2/10
16320/16320 [==============================] - 13s - loss: 0.6934 - acc: 0.5026    
Epoch 3/10
16320/16320 [==============================] - 13s - loss: 0.6933 - acc: 0.4928    
Epoch 4/10
16320/16320 [==============================] - 13s - loss: 0.6932 - acc: 0.4964    
Epoch 5/10
16320/16320 [==============================] - 13s - loss: 0.6932 - acc: 0.4984    
Epoch 6/10
16320/16320 [==============================] - 13s - loss: 0.6932 - acc: 0.5014    
Epoch 7/10
16320/16320 [==============================] - 14s - loss: 0.6933 - acc: 0.4919    
Epoch 8/10
16320/16320 [==============================] - 14s - loss: 0.6932 - acc: 0.5029    
Epoch 9/10
16320/16320 [==============================] - 14s - loss: 0.6932 - acc: 0.4994    
Epoch 10/10
4080/4080 [==============================] - 1s     
Epoch 1/10
16320/16320 [==============================] - 14s - loss: 0.7038 - a

In [17]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

# Evaluate model A
seed = 7
estimator = KerasClassifier(build_fn=create_model_A, epochs=10, batch_size=5, verbose=1)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X_10200_10200_combo_sample_number_50_train, Y_10200_10200_combo_sample_number_50_train, cv=kfold)

print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/10
16320/16320 [==============================] - 15s - loss: 0.7004 - acc: 0.4936    
Epoch 2/10
16320/16320 [==============================] - 15s - loss: 0.6935 - acc: 0.5029    
Epoch 3/10
16320/16320 [==============================] - 15s - loss: 0.6933 - acc: 0.4936    
Epoch 4/10
16320/16320 [==============================] - 15s - loss: 0.6932 - acc: 0.5031    
Epoch 5/10
16320/16320 [==============================] - 15s - loss: 0.6932 - acc: 0.4991    
Epoch 6/10
16320/16320 [==============================] - 15s - loss: 0.6932 - acc: 0.4935    
Epoch 7/10
16320/16320 [==============================] - 15s - loss: 0.6932 - acc: 0.5007    
Epoch 8/10
16320/16320 [==============================] - 15s - loss: 0.6932 - acc: 0.4974    
Epoch 9/10
16320/16320 [==============================] - 15s - loss: 0.6932 - acc: 0.5039    
Epoch 10/10
16320/16320 [==============================] - 15s - loss: 0.6998 - acc: 0.5010    
Epoch 2/10
16320/16320 [=========================